## 머신러닝을 이용한 언어 감지 서비스 구축

### 1. 연구 목표 설정

- 유사서비스 : 파파고, 구글 번역
- 개요
  - 번역 서비스 중 언어 감지 파트는 머신러닝의 지도학습법 중 분류를 사용하겠다
  - 알파벳을 사용하는 영어권에서는 알파벳 언어별로 알파벳의 사용 빈도가 다르다
- 조건
   - 비 영어권은 개별 방법론(완성형(utf-8), 조합형(euc-kr) 코드를 이용하여 판단)
   - 임시값(100byte) 이내 문자열을 배제, 임시값의 임계값은 변경될 수 있다
   - 번역서비스는 딥러닝의 RNN 을 활용하여 처리하는데 여기서는 배제, 단 파파고 API를 활용하여 유사하게 구현
   - 서비스가 오픈하고 데이터가 축적되면 모델을 갱신(언어는 진화하니까) 
       모델을 다시 학습하고 교체를 진행하는데 원활하게 수행되게끔 처리 ( 전략이 필요 ) 일단 여기서는 데이터 축적.

|No|단계|내용|
|:---:|:---|:---|
|1|연구 목표 설정|- 웹서비스<br>- 사용자가 입력한 텍스트를 예측하여 어떤 언어인지 판독한다(영어권.알파벳 사용국)<br>- 머신러닝의 지도학습-분류를 사용하겠다<br>- 파이프라인구축, 하이퍼파라미터튜닝을 이용한 최적화 부분은 제외<br>- 정량적인 목표치는 생략(평가배제)<br>- 임시값(100byte) 이내 문자열을 배제<br>- 논문을 통한 주장의 근거를 체크 |
|2|데이터 획득/수집|- 실전:다양한 텍스트를 수집, 위키피디아, 법률, 소설등등<br>- 구현:제공데이터를 사용(법령/대본/소설등)|
|3|데이터 준비/통찰/전처리|- 알파벳을 제외한 모든 문자 제거(전처리,정규식)<br>- 텍스트를 알파벳의 출현 빈도로 계산한다(문자계산, 데이터의 수치화)<br>- 데이터는 훈련데이터(훈련:50,검증:25)와 테스트데이터(25)로 나눈다 (훈련:테스트=75:25 황금비율 절대적이진 않음)|
|4|데이터 탐색/통찰/시각화|- 논문의 주장을 증명<br>- 영어권 언어별로 알파벳 출현 빈도가 다르다는 명제를 증명/확인<br>- EDA 분석(시각화)를 이용하여 확인, 선형차트, 바차트 등을 활용|
|5|데이터 모델링 및 구축|- 알고리즘 선정<br>- 학습데이터/테스트데이터 준비<br>- 학습<br>- 예측<br>- 성능평가(학습법,하위 카테고리 까지 검토 평가)<br>- 파이프라인구축을 통하여 알고리즘 체인을 적용, 최적의 알고리즘 조합을 찾는다<br>- 연구목표에 도착할때까지 반복|
|6|시스템 통합|- 모델 덤프(학습된 알고리즘을 파일로 덤프)<br>- 웹서비스 구축(Flask 간단하게 구성)<br>- 서비스구축<br>- 모델의 업그레이드를 위한 시스템 추가 <br>- 선순환구조를 위한 번역 요청 데이터의 로그 처리 -> 배치학습,온라인학습 등으로 연결되어 완성|

### 2. 데이터 획득

- 실전:다양한 텍스트를 수집, 위키피디아, 법률, 소설등등
  - 라이브러리 : request, bs4 <- lv3
  - 사이트 : https://언어코드.wikipedia.org/wiki/검색어
       - 예) https://en.wikipedia.org/wiki/Bong_Joon-ho
- 구현:제공데이터를 사용(법령/대본/소설등)

In [47]:
# 모듈 가져오기
import urllib.request as req
from bs4 import BeautifulSoup

In [48]:
# 데이터 획득 사이트
target_site = 'https://{na_code}.wikipedia.org/wiki/{keyword}'.format( na_code='en', keyword='Bong_Joon-ho' )
target_site

'https://en.wikipedia.org/wiki/Bong_Joon-ho'

In [49]:
# 요청 및 SOUP 생성(DOM 트리)
# html5lib 파서 사용 이유 : 대량의 html을 파싱하기 위해 안전성 고려
soup = BeautifulSoup(req.urlopen(target_site),'html5lib')

In [50]:
# 데이터 추출
# css selector : #mw-content-text p
tmp = soup.select('#mw-content-text p')
len(tmp), type(tmp)

(22, list)

In [52]:
# p 밑에 있는 모든 텍스트를 리스트에 모아 둔다 => 멤버수가 22개
# 리스트 생성
texts = []  #list() 와 [] 는 약간의 차이가 있음

for p in tmp :
    # 멤버 추가
    # append 와 extend 의 차이 => 동료와 자식의 차이
    texts.append(p.text)
    
len(texts),texts[:2]

(22,
 ['\n',
  'Bong Joon-ho (Korean:\xa0봉준호, Korean pronunciation:\xa0[poːŋ tɕuːnho → poːŋdʑunɦo]; born September 14, 1969) is a South Korean film director and screenwriter. He garnered international acclaim for his second feature film Memories of Murder (2003), before achieving commercial success with his subsequent films The Host (2006) and Snowpiercer (2013), both of which are among the highest-grossing films of all time in South Korea.[1] \n'])

In [53]:
# 리스트 내포
texts = [p.text for p in tmp]
len(texts), texts[-1:]

(22, [' Media related to Bong Joon-ho at Wikimedia Commons\n'])

In [54]:
# 수집한 데이터를 한개의 텍스트 덩어리로 통합
str_txt=''.join(texts)
len(str_txt), str_txt[:100]

(13245,
 '\nBong Joon-ho (Korean:\xa0봉준호, Korean pronunciation:\xa0[poːŋ tɕuːnho → poːŋdʑunɦo]; born September 14, 19')

In [55]:
# 정규식을 활용하여 알파벳만 남긴다
import re

In [56]:
p = re.compile('[^a-zA-Z]*')

In [57]:
tmp = p.sub('',str_txt)
# 전부 소문자로 처리
tmp.lower()

'bongjoonhokoreankoreanpronunciationpotunhopodunobornseptemberisasouthkoreanfilmdirectorandscreenwriterhegarneredinternationalacclaimforhissecondfeaturefilmmemoriesofmurderbeforeachievingcommercialsuccesswithhissubsequentfilmsthehostandsnowpiercerbothofwhichareamongthehighestgrossingfilmsofalltimeinsouthkoreatwoofhisfilmshavescreenedincompetitionatthecannesfilmfestivalokjawhichpremieredatthecannesfilmfestivalandparasitewhichwonthepalmedoratthecannesfilmfestivalhebecamethefirstkoreandirectortowinthepalmedorparasitealsowonbestforeignlanguagefilmatthethgoldenglobeawardswithbongnominatedforbestdirectorandbestscreenplayforhisworkinmetacriticrankedbongsixteenthonitslistofthebestfilmdirectorsofthestcenturyhisfilmsfeaturetimelysocialthemesgenremixingblackhumorandsuddenmoodshiftsbongjoonhowasbornindaeguinanddecidedtobecomeafilmmakerwhileinmiddleschoolbongjoonhohadahighlyintellectualupbringinghisfatherbongsanggyunisagraphicdesignerandhismaternalgrandfatherparktaewonwasanotedauthorfamousforadayin

### 3. 데이터 준비
- 알파벳을 제외한 모든 문자 제거(전처리,정규식)
  - 편의상 앞단계에서 같이 병행 처리했다
  - 제공 데이터에서는 여기서 실제로 처리하겠다
- 텍스트를 알파벳의 출현 빈도로 계산한다(문자계산, 데이터의 수치화)
- 데이터는 훈련데이터(훈련:50,검증:25)와 테스트데이터(25)로 나눈다 (훈련:테스트=75:25 황금비율 절대적이진 않음)

In [58]:
# 데이터를 다 읽어 들인다 => 파일목록이 필요하다.
# glob 특정 패턴을 부여해서 특정 위치의 파일 목록을 가져온다 => 외장함수
import os.path, glob

In [59]:
file_list = glob.glob('./data/train/*.txt')
len(file_list), type(file_list), file_list[:2]

(20, list, ['./data/train\\en-1.txt', './data/train\\en-2.txt'])

In [60]:
# 파일을 읽어서 알파벳 별 빈도를 계산, 그런 데이터가 en 혹은 fr 등 이 데이터가 어떤 언어인지(레이블, 정답)까지 데이터화 한다
# 1. 파일명 획득
fName = file_list[0]
fName

'./data/train\\en-1.txt'

In [61]:
# 2. 파일명 획득
name = os.path.basename(fName)
name

'en-1.txt'

In [62]:
# 3. 파일명에는 정답(레이블)이 들어있다. 이것을 추출
# 확장성을 고려하여 정규식으로 처리
p = re.compile('^[a-z]{2,}')
if p.match(name):
    lang = p.match(name).group()
else :
    print('일치하지 않음')
lang    

'en'

In [63]:
# 4. 알파벳 빈도 계산
# 4-1. 파일을 읽는다
file = open('./data/train/en-1.txt','r',encoding='utf-8')
# print(file)
train_file = file.readlines()
# print(train_file)
# 4-2. 알파벳만 남긴다(정규식으로 전처리)
str_txt=''.join(train_file)
p = re.compile('[^a-zA-Z]*')
tmp = p.sub('',str_txt)
type(tmp)
# # 4-3. 소문자로 처리
tmp_1 = tmp.lower()
tmp_1
# 4-4. 알파벳 별로 카운트 계산 = > 효율적인 로직, 방법으로 만들기
alpha_count = 0

for line in tmp_1:
    alpha_count += [c.isalpha() for c in line].count(True)
alpha_count    

4595

In [ ]:
alpha_count = 0

for line in open('./data/train/en-1.txt','r',encoding='utf-8'):
    alpha_count += [c.isalpha() for c in line].count(True)
alpha_count   

In [ ]:
from collections import Counter
result = Counter(tmp_1)
print(result)

In [ ]:
file_list = glob.glob('./data/train/*.txt')

all_list = []

for list in file_list:
    f = open(list,'r',encoding='utf-8')    
    all_list.extend(f.readlines())
    
str_txt=''.join(all_list)    

p = re.compile('[^a-zA-Z]*')

tmp = p.sub('',str_txt)

tmp_1 = tmp.lower()

result = Counter(tmp_1)

print(result)     

In [64]:
# 4. 알파벳 빈도 계산
# with : 입출력(I/O)계열에서 사용, close를 자동으로 처리
with open(fName, 'r', encoding = 'utf-8') as f:
    # 4-1. 파일을 읽는다.
#     print(f.read())
    # 4-3. 소문자로 처리
    text = f.read().lower()

    # 4-2. 알파벳만 남긴다(정규식으로 전처리)
    #정규식을 필요에 의해서 보정이 더 가능
    p =re.compile('[^a-z]')   # * 있으나 없으나 똑같음
    text = p.sub('',text)
#     print(text)
    
# #     pass

len(text), text[:2], type(text)
# text

(4595, 'th', str)

In [65]:
# 4-4. 알파벳 별로 카운트 계산=>효율적인 로직, 방법이 필요
# 알파벳 한개 한개를 카운트해서 담을 그릇(공간) -> 순서가 존재
# 알파벳은 개수가? 26개
counts = [ 0 for n in range(26) ]
# a=>0, b=>1, ... z=>25 알파벳순으로 리스트의 맴버 인덱스를 적용
# -> 한문자씩 읽는다
# ord('a') = a라는 문자열의 아스키값을 리턴 => 97
# "https://namu.wiki/w/아스키 코드" 참조
for word in text:
    #print('문자열 한개', word )
    #print('해당 문자열의 카운트 개수를 담고 있는 리스트상의위치', ord(word)-ord('a'))
    #print('해당 문자열의 카운트수', counts[ord(word)-ord('a')]  )
    # 구현
    counts[ord(word)-ord('a')] += 1
    #break
# 원본, 리스트의 맴버들의 카운트 총합 == 문자열의수
print(counts), sum(counts), len(text), min(counts), max(counts)
# 카운트 데이터를 들여다 보니 값의 편차가 크다 => 학습효과가 저하 => 값을 특정구간으로 재배치 => 정규화(nomarlize) 0~1 사이로
# a빈도/총빈도 .... z빈도/총빈도

[349, 59, 210, 212, 484, 72, 88, 201, 340, 8, 25, 247, 121, 356, 412, 76, 0, 357, 282, 370, 119, 45, 65, 3, 92, 2]


(None, 4595, 4595, 0, 484)

In [66]:
# 정규화 수행하기 
# 연속 데이터 타입의 멤버들을 하나씩 접근해서 작업을 한다 => map 함수 사용
total_count = sum(counts)
freq = list( map( lambda x:x/total_count, counts))
freq[:2], len(freq), sum(freq)

TypeError: 'str' object is not callable

In [67]:
# 파일 경로를 넣으면 정답(국가코드), 알파벳빈도 를 계산한 데이터를 리턴하는 함수 구현
def detect_trans_lang_freq(file_path):
    # 파일명 획득
    name = os.path.basename(file_path)
    # 정답, 국가코드 획득
    p = re.compile('^[a-z]{2,}')
    if p.match(name):
        lang = p.match(name).group()
    else :
        return None, None
    #---------------------------------
    with open( file_path, 'r', encoding='utf-8' ) as f:  # 파일오픈
        text = f.read().lower()             # 전부 읽어서 소문자로 리턴
        p    = re.compile('[^a-z]')         # 정규식(알파벳소문자만제외)
        text = p.sub( '' , text )           # 소문자만 남는다
        counts = [ 0 for n in range(26) ]  # 알파벳별 카운트를 담을 공간(리스트)
        limit_a = ord('a')                  # 매번 반복해서 작업하니까 그냥 최초 한번 변수로 받아서 계속사용
        for word in text:
            counts[ord(word)-limit_a] += 1  # 문자 1개당 카운트 추가
    # 빈도수는 값이 너무 퍼져있기때문에 0~1 사이로 정규화 시켜준다. => 학습효과가 뛰어나진다
        total_count = sum(counts)
        freq = list(map( lambda x:x/total_count , counts))
    return lang, freq

print(detect_trans_lang_freq(fName))

TypeError: 'str' object is not callable

In [ ]:
# train을 입력으로 넣으면 훈련용 데이터를 싹다 뽑아온다 (<-> test)
def load_data(style='train'):
    langs = []
    freqs = []
    file_list = glob.glob('./data/%s/*.txt' % style )
    for file in file_list:
        lang, freq = detect_trans_lang_freq(file)
        langs.append(lang)
        freqs.append(freq)
    # 딕셔너리 정적 구성으로 최종 데이터 형태를 맞춰준다 => Dataframe 형태 고려
    return {'labels':langs, 'freqs':freqs }
load_data()['labels'][:2], load_data()['freqs'][:2][0]

In [ ]:
# 훈련용
train_data = load_data()
# 테스트용
test_data = load_data('test')

### 4. 데이터 탐색

### 5. 데이터 모델링 및 모델 구축

### 6. 시스템 통합